This code will use a selenium web crawler to search ESRB.org for  videogame ratings that appear as NaN in the original data set. ESRB ratings that cannot be found through the site will be marked as Unknown. The webcrawler code can be found in search_esrb.py. Multiprocessing was used to run multiple webdrivers at once. The resulting information will be used in Videogame KNN notebook.

In [1]:
import pandas as pd

In [2]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
data = pd.read_csv(r"vgsales.csv")
data.head()

,Unnamed: 0,Rank,Name,Genre,ESRB_Rating,Platform,Publisher,Developer,Critic_Score,User_Score,Total_Shipped,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year
0,0,1,Wii Sports,Sports,E,Wii,Nintendo,Nintendo EAD,7.7,NaN,82.86,NaN,NaN,NaN,NaN,NaN,2006.0
1,2,3,Mario Kart Wii,Racing,E,Wii,Nintendo,Nintendo EAD,8.2,9.1,37.14,NaN,NaN,NaN,NaN,NaN,2008.0
2,4,5,Wii Sports Resort,Sports,E,Wii,Nintendo,Nintendo EAD,8.0,8.8,33.09,NaN,NaN,NaN,NaN,NaN,2009.0
3,5,6,Pokemon Red / Green / Blue Version,Role-Playing,E,GB,Nintendo,Game Freak,9.4,NaN,31.38,NaN,NaN,NaN,NaN,NaN,1998.0
4,6,7,New Super Mario Bros.,Platform,E,DS,Nintendo,Nintendo EAD,9.1,8.1,30.80,NaN,NaN,NaN,NaN,NaN,2006.0


First, we will drop columns that won't be used in the KNN data analysis

In [4]:
data=data.drop(['User_Score','Critic_Score','NA_Sales','PAL_Sales','JP_Sales','Other_Sales','Unnamed: 0','Global_Sales'],axis=1)
data.head()

,Rank,Name,Genre,ESRB_Rating,Platform,Publisher,Developer,Total_Shipped,Year
0,1,Wii Sports,Sports,E,Wii,Nintendo,Nintendo EAD,82.86,2006.0
1,3,Mario Kart Wii,Racing,E,Wii,Nintendo,Nintendo EAD,37.14,2008.0
2,5,Wii Sports Resort,Sports,E,Wii,Nintendo,Nintendo EAD,33.09,2009.0
3,6,Pokemon Red / Green / Blue Version,Role-Playing,E,GB,Nintendo,Game Freak,31.38,1998.0
4,7,New Super Mario Bros.,Platform,E,DS,Nintendo,Nintendo EAD,30.80,2006.0


Next, we will collect the Names of videogames that have a NaN ESRB Rating. Rank is collected as it is unique thus can be used as an index later.

In [5]:
missing_esrb=data[['Rank','Name']].loc[data['ESRB_Rating'].isna()]

Next, we will import selenium and multiprocessing to develope our webcrawler. Search_esrb has to be imported as a .py script to run multiprocessing as multiprocessing cant run a function thru jupyter or IDLE. Search_esrb is coded as follows:



In [6]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains 
from multiprocessing import Pool
import search_esrb


We split our missing data into chunks to feed our pool. A num number of chrome search engines will open to collect the esrb rating data. The process takes 3 hours.

In [7]:
tot = missing_esrb.shape[0]
num =6
val = tot//num
chunks =[missing_esrb[0:val], missing_esrb[num*val:tot]]
for i in range(num-1):
    chunks.append(missing_esrb[(i+1)*val:(i+2)*val])
    
if __name__ ==  '__main__': 
    print('starting')
    p = Pool(processes=num-1)
    updated=p.map(search_esrb.search, chunks)
        

starting


search_esrb returns a list of tuples containing the Rank, Name, and ESRB_Rating of the videogames in missing_esrb. The tuples are converted into one dataframe.

In [8]:
cols=['Rank','Name','ESRB_Rating']
df = pd.DataFrame(updated[0], columns = cols )
for i in range(num):
    df_new =pd.DataFrame(updated[i+1],columns=cols)
    df=df.append(df_new)
df = df.reset_index(drop=True)

Since the indices do not match, we will use Rank as the unique index to combine our data sets

In [23]:
test=data
test=test.set_index('Rank').combine_first(df.set_index('Rank'))
print('The number of missing esrb are now : ' + str(test.loc[test['ESRB_Rating'].isna()].shape[0]))




The number of missing esrb are now : 0


Lastly, we change E10plus to its equivalent E10 in the orginal data and save the file to use in the Videogame KNN notebook.

In [10]:

test['ESRB_Rating'].unique()


array(['E', 'Unknown', 'M', 'E10', 'T', 'E10plus', 'RP', 'EC', nan, 'AO',
       'KA'], dtype=object)

In [11]:
test['ESRB_Rating'].loc[test['ESRB_Rating']=='E10plus'] = 'E10'

C:\Users\laure\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
test.to_csv(r'C:\Users\laure\updated_videogames.csv')